## Defining modules/packages to import

In [3]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy

# importing package to recognize stop words i.e. the, and, an etc
from spacy.lang.en.stop_words import STOP_WORDS
stop_words = spacy.lang.en.stop_words.STOP_WORDS

import datetime

import plotly.express as px

#nltk.download('punkt')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

stemmer = PorterStemmer()

In [4]:
# importing module to ignore the warnings
import warnings
warnings.simplefilter('ignore')

## Defining functions to be used

In [6]:
#### Now creating some functions to do text processing
# Removing hashtags and mentions
def get_hashtags(text):
    hashtags = re.findall(r'\#\w+',text.lower())
    return hashtags
def get_mentions(text):
    mentions = re.findall(r'\@\w+',text.lower())
    return mentions

# Cleaning up the text of the tweets
def remove_content(text):
    text = re.sub(r"http\S+", "", text) #remove urls
    text=re.sub(r'\S+\.com\S+','',text) #remove urls
    text=re.sub(r'\@\w+','',text) #remove mentions
    text =re.sub(r'\#\w+','',text) #remove hashtags
    return text

def process_tweet(tweet):
    """ 
    tweets cleaning by 
    1) lowering the case of the tweet, 
    2) removing unwanted symbols and replacing them with a whitespace, 
    3) split sentences into words according to whitespaces and then 
    4) join back with a single whitespace as separator between various words
    """
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

def process_text(text, stem=False): #clean text
    text=remove_content(text)
    lemmatizer=WordNetLemmatizer()
    text = re.sub('[^A-Za-z]', ' ', text.lower()) #remove non-alphabets
    text = re.sub(r"\bamp\b", ' ', text.lower()) #remove "amp" which is coming from the translation of &
    text = re.sub(r"\bco\b", ' ', text.lower()) #remove "co" which was one of the top words found below
    tokenized_text = word_tokenize(text) #tokenize
    #tokenized_text = [lemmatizer.lemmatize(word) for word in tokenized_text]
    clean_text = [
         word for word in tokenized_text
         if (word not in stop_words and len(word)>1)
    ]
    if stem:
        clean_text=[stemmer.stem(word) for word in clean_text]
    clean_text = [lemmatizer.lemmatize(word) for word in clean_text]
    return ' '.join(clean_text)

#functions used to remove search terms from all the tweets
#function to remove duplicates from a string - in this case the string is the keywords used to scrape the tweets
def removeDupWithoutOrder(string): 
    words = string.lower().split()
    return " ".join(sorted(set(words), key=words.index)).replace('OR', '').replace('  ', ' ')

#function to search for string (i.e. tweet in this case) and remove specific words (search_terms in this case)
def remove_search(text, search_terms):
    query = text.lower()
    querywords = query.split()
    resultwords  = [word for word in querywords if word.lower() not in search_terms]
    return ' '.join(resultwords)

# define function to plot frequency of bi-grams, tri-grams, single words, phrases etc
from sklearn.feature_extraction.text import CountVectorizer
def plot_topn(sentences, ngram_range=(1,3), top=20,firstword=''):
    c=CountVectorizer(ngram_range=ngram_range)
    X=c.fit_transform(sentences)
    words=pd.DataFrame(X.sum(axis=0),columns=c.get_feature_names()).T.sort_values(0,ascending=False).reset_index()
    res=words[words['index'].apply(lambda x: firstword in x)].head(top)
    pl=px.bar(res, x='index',y=0)
    pl.update_layout(yaxis_title='count',xaxis_title='Phrases')
    pl.show('png')

## Search terms used to do tweet scraping

In [7]:
keywords = ["UK asexual", "UK bisexual OR UK bisex OR UK bi-sexual", "UK drag","UK dyke", "UK faggot",
                 "UK gay", "UK gender","UK genderfluid","UK homophobia OR UK homophobic","UK intersex", "UK lesbo", "UK lesbian", 
                 "UK non-binary OR UK nonbinary","UK omnigender","UK pansexual","UK polysexual", 
                 "UK pride", "UK queer", "UK sogie", "UK trans","UK unisexual", "UK ursula", "UK transphobic"]
keywords_concat = ' '.join(keywords)

search_concatenation = keywords_concat
# now creating a string with unique occurrences of words from "search_concatenation"
search_concat_unique = removeDupWithoutOrder(search_concatenation).lower()

In [8]:
search_concat_unique

'uk asexual bisexual or bisex bi-sexual drag dyke faggot gay gender genderfluid homophobia homophobic intersex lesbo lesbian non-binary nonbinary omnigender pansexual polysexual pride queer sogie trans unisexual ursula transphobic'

## Now importing database to do analysis

In [ ]:
# importing scraped database and putting it in df, taking only the columns below
df=pd.read_csv('../data/lgbtq_df_2019_2020.csv')[['date','tweet','nlikes','nreplies','nretweets']]

In [ ]:
df.head(5)

In [ ]:
# taking the 'date' column (which is a string) and extracting date, times, breaking it down in months, years etc

df['INCDTTM'] =  pd.to_datetime(df['date'], infer_datetime_format=True)
df['Time'] = [datetime.datetime.time(d) for d in df['INCDTTM']] 
df['Date'] = [datetime.datetime.date(d) for d in df['INCDTTM']]
df['Year'] = [datetime.datetime.date(d).year for d in df['INCDTTM']] 
df['Month'] = [datetime.datetime.date(d).month for d in df['INCDTTM']] 
df['Day'] = [datetime.datetime.date(d).day for d in df['INCDTTM']] 
df['Hours'] = [datetime.datetime.time(d).hour for d in df['INCDTTM']] 
df['Minutes'] = [datetime.datetime.time(d).minute for d in df['INCDTTM']] 
df['Seconds'] = [datetime.datetime.time(d).second for d in df['INCDTTM']]
df = df.drop(columns = ['date','INCDTTM'])
df.head()

In [ ]:
# removing useless content (hashtags, mentions)
df['tweet']=df['tweet'].apply(lambda x: remove_content(x))

In [ ]:
# Several functions applied here: processing the *tweet* to remove punctuation, hashtags, mentions
# then removing the search terms used to do the tweets scraping and re-process the *text* 

df['cleaned_tweets']=df['tweet'].apply(lambda x: process_tweet(x))

#this next row drops from the scraped tweets the same keywords that were used to perform the search.
#if these words are excluded, the sentiment analysis performed with text blob looks at the whole tweet 
#and can spit out a different sentiment wrt if leaving the search terms in. Perhaps worth test it out a bit
#and look at TextBlob with naivebayes too.

#df['cleaned_tweets']=df['cleaned_tweets'].apply(lambda x: remove_search(x, search_concat_unique))

df['cleaned_tweets']=df['cleaned_tweets'].apply(lambda x: process_text(x))

In [ ]:
df.head()

In [ ]:
# taking the cleaning tweets and dividing them using the space - cleaned tweets contain tweets that are stemmed and cleaned
tweet_list = df['cleaned_tweets'].tolist()
all_words=' '.join(tweet_list).split()

In [ ]:
plot_topn(tweet_list, ngram_range=(1,1))

In [ ]:
plot_topn(tweet_list, ngram_range=(2,2))

In [ ]:
plot_topn(tweet_list, ngram_range=(3,3))

## Wordcloud of most used words

In [ ]:
from wordcloud import WordCloud, STOPWORDS
temp=' '.join(df['cleaned_tweets'].tolist())
wordcloud = WordCloud(width = 800, height = 500, 
                background_color ='white',
                min_font_size = 10).generate(temp)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off")
plt.tight_layout(pad = 0) 
  
plt.show('png')

## Sentiment analysis

In [ ]:
from textblob import TextBlob 

def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(tweet) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'    

In [ ]:
df.head()

In [ ]:
df['sentiment']=df['cleaned_tweets'].apply(lambda x: get_tweet_sentiment(x))

In [ ]:
df.head()

In [ ]:
df.sentiment.value_counts()

In [ ]:
df['Positive']= df['sentiment'].apply(lambda x: 1 if (x=="positive")  else 0)
df['Negative']= df['sentiment'].apply(lambda x: 1 if (x=="negative")  else 0)
df['Neutral']= df['sentiment'].apply(lambda x: 1 if (x=="neutral")  else 0)

df.head()

In [ ]:
df_timeline = df.groupby(df.Date).sum().reset_index()
df_timeline['Total'] = df_timeline.Positive + df_timeline.Negative + df_timeline.Neutral
df_timeline

In [ ]:
import numpy as np
from datetime import datetime, timedelta

t = np.arange(df_timeline.Date[0], df_timeline.Date[365], timedelta(days=7)).astype(datetime)

In [ ]:
#plt.figure(figsize=(20,5))
#plt.bar(df_timeline.Date, (df_timeline['Positive']/df_timeline['Total']));
#plt.bar(df_timeline.Date, (df_timeline['Negative']/df_timeline['Total']), bottom = (df_timeline['Positive']/df_timeline['Total']));
#plt.legend(['Positive', 'Negative']);
#plt.xticks(t, rotation='vertical');
#plt.ylabel('Number of positive and negative tweets');
#plt.xlabel('Date');

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_bar(x=df_timeline.Date, y=100*df_timeline.Negative/(df_timeline.Total), name="% Negative")
fig.add_bar(x=df_timeline.Date, y=100*df_timeline.Positive/(df_timeline.Total), name="% Positive")
fig.add_bar(x=df_timeline.Date, y=100*df_timeline.Neutral/df_timeline.Total, name="% Neutral")
fig.add_trace(go.Scatter(x=[df_timeline.Date[0], df_timeline.Date[365]],y=[50,50],name="50% threshold"))
fig.update_layout(barmode="relative")
fig.update_layout(showlegend=True,title="Tweets split by sentiment: % positive, % negative and % neutral",
                 yaxis_title="% of total tweets", xaxis_title="Time")
fig.show()

### Some further analyses on percentages of positive/negative/neutral tweets and saving them locally

In [ ]:
df_new = df_timeline[['Date', 'Negative','Positive','Neutral','Total']].copy()

In [ ]:
df_new['Percentage_neg'] = 100*(df_new.Negative/df_new.Total)
df_new['Percentage_pos'] = 100*(df_new.Positive/df_new.Total)
df_new['Percentage_neutr'] = 100*(df_new.Neutral/df_new.Total)
df_new.head()

In [ ]:
df_new.shape

In [ ]:
#df_new.to_csv('../data/lgbtq_2019_2020_postprocessed.csv', index=False)

### Now we turn our attention to the reactions and engagements - by categories: likes, replies, retweets. What is making more 'noise'? Negative or positive tweets?

In [ ]:
reactions_vert = pd.DataFrame(columns=['Date', 'Positive_nlikes', 'Positive_nreplies', 'Positive_nretweets', 'Negative_nlikes', 'Negative_nreplies', 'Negative_nretweets', 'Neutral_nlikes', 'Neutral_nreplies', 'Neutral_nretweets', 'Total_nlikes', 'Total_nreplies', 'Total_nretweets'])

In [ ]:
reactions_vert.Date = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().Date
reactions_vert.Positive_nlikes = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_vert.Positive_nreplies = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_vert.Positive_nretweets = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nretweets

reactions_vert.Negative_nlikes = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_vert.Negative_nreplies = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_vert.Negative_nretweets = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nretweets

reactions_vert.Neutral_nlikes = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_vert.Neutral_nreplies = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_vert.Neutral_nretweets = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nretweets

#cumulative total of likes, replies and retweets per day

reactions_vert.Total_nlikes = reactions_vert.Positive_nlikes + reactions_vert.Negative_nlikes + reactions_vert.Neutral_nlikes
reactions_vert.Total_nreplies = reactions_vert.Positive_nreplies + reactions_vert.Negative_nreplies + reactions_vert.Neutral_nreplies
reactions_vert.Total_nretweets = reactions_vert.Positive_nretweets + reactions_vert.Negative_nretweets + reactions_vert.Neutral_nretweets

In [ ]:
reactions_vert.head()

In [ ]:
reactions_vert.shape

In [ ]:
reactions_vert.to_csv('../data/lgbtq_2019_2020_reactions_vert.csv', index=False)

In [ ]:
reactions_perc_v = pd.DataFrame(columns=['Date', 'Positive_nlikes', 'Positive_nreplies', 'Positive_nretweets', 'Negative_nlikes', 'Negative_nreplies', 'Negative_nretweets', 'Neutral_nlikes', 'Neutral_nreplies', 'Neutral_nretweets'])

reactions_perc_v.Date = reactions_vert.Date

# Now defining percentages of each reaction (like or reply or retweet) wrt to their total across sentiment. 
# By doing so we're calculating how each reaction is split by **sentiment** 

reactions_perc_v.Positive_nlikes = 100*reactions_vert.Positive_nlikes/reactions_vert.Total_nlikes
reactions_perc_v.Positive_nreplies = 100*reactions_vert.Positive_nreplies/reactions_vert.Total_nreplies
reactions_perc_v.Positive_nretweets = 100*reactions_vert.Positive_nretweets/reactions_vert.Total_nretweets

reactions_perc_v.Negative_nlikes = 100*reactions_vert.Negative_nlikes/reactions_vert.Total_nlikes
reactions_perc_v.Negative_nreplies = 100*reactions_vert.Negative_nreplies/reactions_vert.Total_nreplies
reactions_perc_v.Negative_nretweets = 100*reactions_vert.Negative_nretweets/reactions_vert.Total_nretweets

reactions_perc_v.Neutral_nlikes = 100*reactions_vert.Neutral_nlikes/reactions_vert.Total_nlikes
reactions_perc_v.Neutral_nreplies = 100*reactions_vert.Neutral_nreplies/reactions_vert.Total_nreplies
reactions_perc_v.Neutral_nretweets = 100*reactions_vert.Neutral_nretweets/reactions_vert.Total_nretweets

reactions_perc_v['Positive tweets'] = 100*df_timeline.Positive/df_timeline.Total
reactions_perc_v['Negative tweets'] = 100*df_timeline.Negative/df_timeline.Total
reactions_perc_v['Neutral tweets'] = 100*df_timeline.Neutral/df_timeline.Total

reactions_perc_v.head()

In [ ]:
reactions_perc_v.to_csv('../data/lgbtq_2019_2020_reactions_perc_v.csv', index=False)

### Plots showing for each type of reactions, what's the split by sentiment - first by % then by absolute numbers

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Positive_nlikes, name="% Positive", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Negative_nlikes, name="% Negative", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Neutral_nlikes, name="% Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Likes' split by sentiment",
                 yaxis_title="% of likes", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Positive_nreplies, name="% Positive", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Negative_nreplies, name="% Negative", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Neutral_nreplies, name="% Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Replies' split by sentiment",
                 yaxis_title="% of replies", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Positive_nretweets, name="% Positive", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Negative_nretweets, name="% Negative", row=1, col=1)
fig.add_bar(x=reactions_perc_v.Date, y=reactions_perc_v.Neutral_nretweets, name="% Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Retweets' split by sentiment",
                 yaxis_title="% of retweets", xaxis_title="Time")
fig.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Positive_nlikes, name="Positive", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Negative_nlikes, name="Negative", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Neutral_nlikes, name="Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Likes' split by sentiment",
                 yaxis_title="Number of likes", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Positive_nreplies, name="Positive", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Negative_nreplies, name="Negative", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Neutral_nreplies, name="Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Replies' split by sentiment",
                 yaxis_title="Number of replies", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Positive_nretweets, name="Positive", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Negative_nretweets, name="Negative", row=1, col=1)
fig.add_bar(x=reactions_vert.Date, y=reactions_vert.Neutral_nretweets, name="Neutral", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="'Retweets' split by sentiment",
                 yaxis_title="Number of retweets", xaxis_title="Time")
fig.show()


In [ ]:
# Now we create a new dataframe to illustrate how each sentiment is split by reaction

reactions_hor = pd.DataFrame(columns=['Date', 'Positive_nlikes', 'Positive_nreplies', 'Positive_nretweets', 'Negative_nlikes', 'Negative_nreplies', 
                                  'Negative_nretweets', 'Neutral_nlikes', 'Neutral_nreplies', 'Neutral_nretweets', 
                                  'Total_Positive', 'Total_Negative', 'Total_Neutral'])

reactions_hor.Date = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().Date
reactions_hor.Positive_nlikes = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_hor.Positive_nreplies = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_hor.Positive_nretweets = df.loc[df['Positive'] == 1].groupby(df.Date).sum().reset_index().nretweets

reactions_hor.Negative_nlikes = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_hor.Negative_nreplies = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_hor.Negative_nretweets = df.loc[df['Negative'] == 1].groupby(df.Date).sum().reset_index().nretweets

reactions_hor.Neutral_nlikes = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nlikes
reactions_hor.Neutral_nreplies = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nreplies
reactions_hor.Neutral_nretweets = df.loc[df['Neutral'] == 1].groupby(df.Date).sum().reset_index().nretweets

#cumulative total of (likes+replies+retweets) per day by sentiment

reactions_hor.Total_Positive = reactions_hor.Positive_nlikes + reactions_hor.Positive_nreplies + reactions_hor.Positive_nretweets
reactions_hor.Total_Negative = reactions_hor.Negative_nlikes + reactions_hor.Negative_nreplies + reactions_hor.Negative_nretweets
reactions_hor.Total_Neutral = reactions_hor.Neutral_nlikes + reactions_hor.Neutral_nreplies + reactions_hor.Neutral_nretweets

reactions_hor.head()

In [ ]:
reactions_hor.to_csv('../data/lgbtq_2019_2020_reactions_hor.csv', index=False)

In [ ]:
reactions_perc_h = pd.DataFrame(columns=['Date', 'Positive_nlikes', 'Positive_nreplies', 'Positive_nretweets', 
                                         'Negative_nlikes', 'Negative_nreplies', 'Negative_nretweets', 
                                         'Neutral_nlikes', 'Neutral_nreplies', 'Neutral_nretweets'])

reactions_perc_h.Date = reactions_hor.Date

# Now defining percentages of each reaction (like or reply or retweet) wrt to their total across sentiment. 
# By doing so we're calculating how each reaction is split by **sentiment** 

reactions_perc_h.Positive_nlikes = 100*reactions_hor.Positive_nlikes/reactions_hor.Total_Positive
reactions_perc_h.Positive_nreplies = 100*reactions_hor.Positive_nreplies/reactions_hor.Total_Positive
reactions_perc_h.Positive_nretweets = 100*reactions_hor.Positive_nretweets/reactions_hor.Total_Positive

reactions_perc_h.Negative_nlikes = 100*reactions_hor.Negative_nlikes/reactions_hor.Total_Negative
reactions_perc_h.Negative_nreplies = 100*reactions_hor.Negative_nreplies/reactions_hor.Total_Negative
reactions_perc_h.Negative_nretweets = 100*reactions_hor.Negative_nretweets/reactions_hor.Total_Negative

reactions_perc_h.Neutral_nlikes = 100*reactions_hor.Neutral_nlikes/reactions_hor.Total_Neutral
reactions_perc_h.Neutral_nreplies = 100*reactions_hor.Neutral_nreplies/reactions_hor.Total_Neutral
reactions_perc_h.Neutral_nretweets = 100*reactions_hor.Neutral_nretweets/reactions_hor.Total_Neutral

reactions_perc_h['Positive tweets'] = 100*df_timeline.Positive/df_timeline.Total
reactions_perc_h['Negative tweets'] = 100*df_timeline.Negative/df_timeline.Total
reactions_perc_h['Neutral tweets'] = 100*df_timeline.Neutral/df_timeline.Total

reactions_perc_h.head()

In [ ]:
reactions_perc_h.to_csv('../data/lgbtq_2019_2020_reactions_perc_h.csv', index=False)

### Plots showing for each type of sentiment, what's the split by reaction type - first by % then by absolute numbers

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Positive_nlikes, name="% Likes", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Positive_nreplies, name="% Replies", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Positive_nretweets, name="% Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for positive tweets",
                 yaxis_title="% of reactions", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Negative_nlikes, name="% Likes", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Negative_nreplies, name="% Replies", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Negative_nretweets, name="% Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for negative tweets",
                 yaxis_title="% of reactions", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Neutral_nlikes, name="% Likes", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Neutral_nreplies, name="% Replies", row=1, col=1)
fig.add_bar(x=reactions_perc_h.Date, y=reactions_perc_h.Neutral_nretweets, name="% Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for neutral tweets",
                 yaxis_title="% of reactions", xaxis_title="Time")
fig.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Positive_nlikes, name="Likes", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Positive_nreplies, name="Replies", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Positive_nretweets, name="Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for positive tweets",
                 yaxis_title="Number of reactions", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Negative_nlikes, name="Likes", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Negative_nreplies, name="Replies", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Negative_nretweets, name="Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for negative tweets",
                 yaxis_title="Number of reactions", xaxis_title="Time")
fig.show()

fig = make_subplots(rows=1, cols=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Neutral_nlikes, name="Likes", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Neutral_nreplies, name="Replies", row=1, col=1)
fig.add_bar(x=reactions_hor.Date, y=reactions_hor.Neutral_nretweets, name="Retweets", row=1, col=1)
fig.update_layout(barmode="relative",showlegend=True,title="Reactions classification for neutral tweets",
                 yaxis_title="Number of reactions", xaxis_title="Time")
fig.show()
